##Group Members: Gaurav Karwa - R11889434, Harshitha Arava - R11901126, Shweta Sumant Koyande - R11904525, Surendranath Avvaru - R11894279

In [1]:
! pip install nanogcg

In [2]:
!pip install huggingface_hub

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Checking cuda availability

In [4]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


##Note: HF_TOKEN below will be needed to be created for each individual running this notebook. Steps to create are below. Tokens are unique so we cannot provide them here.

The below is for downloading the model from hugging face, you need to request access form hugging face to download it, details can be found on the specifc models page

https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

You will also need Hugging Face token, you can get this from your profile section in hugging face

https://huggingface.co/docs/hub/security-tokens

The above link can be used as a guide

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

# Your hf token goes here
login(token='HF_TOKEN')
torch.use_deterministic_algorithms(False)
# model id for which model you want to run this attack
model_id = "meta-llama/Llama-3.2-3B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)



The below code is for creating datasets from the parquet file (provided in zip folder), using datasets.
The provided dataset has 855k rows, but we are running the nano gcg attack only for 200 rows, as running for all sample takes considerate amount of time.


link for the dataset we used :https://huggingface.co/datasets/Afterglow777/chemical_dpo_dataset


In [7]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download


# Download the specified parquet files to your local machine
local_file_paths = ['/content/train-00004-of-00050.parquet']
ds = load_dataset("parquet", data_files=local_file_paths)

ds_subset = ds['train'].select(range(200))
print(ds_subset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 200
})


Now we will implement the GCG attack and store the results in "results" variable where, we get the best adv string, its loss and each of them are run for 50 iterations or steps, author advised is 250, but we do get decent results with 50 steps and 250 will take too much time to run

In [ ]:
import nanogcg
import torch

from nanogcg import GCGConfig

config = GCGConfig(
    num_steps=50,
    search_width=64,
    topk=64,
    seed=42,
    verbosity="WARNING",
    early_stop=True
)

# Iterate through the dataset
results = []
for row in ds_subset:
    prompt = row['prompt']

    positive_prompt = row['rejected']

    # Messages template
    messages = [
    {"role": "user", "content":  prompt}
    ]
    target = positive_prompt

    gcg_result = nanogcg.run(model, tokenizer, messages, target, config)

    # Extract best_loss and best_string
    best_loss = gcg_result.best_loss
    best_string = gcg_result.best_string

    combined_input = best_string + " " + positive_prompt
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(
                          combined_input,
                          return_tensors="pt",
                          padding=True,
                          truncation=True
                      ).to("cuda")

    # Set model pad_token_id to tokenizer's pad_token_id
    model.config.pad_token_id = tokenizer.pad_token_id

    # Generate attention mask
    attention_mask = inputs.attention_mask
    # inputs = tokenizer(combined_input, return_tensors="pt").to("cuda")

    # Generate inference
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=attention_mask,
            max_length=200,  # Adjust max length as needed
            do_sample=True,
            top_p=0.7,
            temperature=0.6,
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Save the result with context for analysis
    results.append({
        "prompt": prompt,

        "positive_prompt": positive_prompt,
        "best_loss": best_loss,
        "best_string": best_string,
        "Adverserial_reply":generated_text
    })


# Save results to a file




/usr/local/lib/python3.10/dist-packages/transformers/models/llama/modeling_llama.py:158: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
  freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but

Below is to convert the result in pandas data frame and then store it in the csv file for analysis

In [11]:
import pandas as pd
output_path = "output_results.csv"
df = pd.DataFrame(results)
df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")


Results saved to output_results.csv


END OF ATTACK

